In [1]:
from caveclient import CAVEclient, auth
from caveclient import chunkedgraph as cg
# import gcsfs
# from google.cloud import storage
import pandas as pd
import numpy as np
import time
from tqdm import tqdm

In [8]:
url = "https://pcgv3local.brain-wire-test.org"
global_url = "https://global.brain-wire-test.org/"
datastack = "fish1_full"
dataset = "fish1_v250117"
resolution = [16, 16, 30]

In [9]:
client = CAVEclient(datastack_name=datastack, server_address=global_url)
cggraph = cg.ChunkedGraphClient(server_address=url, table_name=dataset, auth_client=auth.AuthClient(token=client.auth.token))
cv = client.info.segmentation_cloudvolume()

In [10]:
cggraph.segmentation_info

{'@type': 'neuroglancer_multiscale_volume',
 'app': {'supported_api_versions': [0, 1]},
 'chunks_start_at_voxel_offset': True,
 'data_dir': 'gs://fish1-pychunkedgraph/seg_241003_241003_restore_all/segmentation',
 'data_type': 'uint64',
 'graph': {'bounding_box': [2048, 2048, 512],
  'chunk_size': [256, 256, 128],
  'cv_mip': 0,
  'n_bits_for_layer_id': 8,
  'n_layers': 12,
  'spatial_bit_masks': {'1': 10,
   '2': 10,
   '3': 9,
   '4': 8,
   '5': 7,
   '6': 6,
   '7': 5,
   '8': 4,
   '9': 3,
   '10': 2,
   '11': 1,
   '12': 1}},
 'mesh': 'graphene_meshes_v250117',
 'mesh_dir': 'graphene_meshes_v250117',
 'mesh_metadata': {'max_meshed_layer': 6,
  'uniform_draco_grid_size': 9,
  'unsharded_mesh_dir': 'dynamic'},
 'num_channels': 1,
 'scales': [{'chunk_sizes': [[128, 128, 64]],
   'compressed_segmentation_block_size': [8, 8, 4],
   'encoding': 'compressed_segmentation',
   'key': 'np.float64(16.0)xnp.float64(16.0)xnp.float64(30.0)',
   'resolution': [16.0, 16.0, 30.0],
   'sharding': {'

In [4]:
# key_path = "/root/.cloudvolume/secrets/fish1-pychunkedgraph-google-secret.json"
# project_id = "engert-goog-connectomics"
# bucket_name = 'fish1-pychunkedgraph'
# folder_name = '241003_restore_all_chosenedges_pychunk_dropped_coords'

In [11]:
def get_latest_root_id(x, y, z):
    supervoxel_id = cv.download_point(pt=(x, y, z), 
                                      size=1, 
                                      agglomerate=False, 
                                      coord_resolution=resolution)
    supervoxel_id = np.int64(supervoxel_id[0, 0, 0, 0]) 
    root_id = cggraph.get_root_id(supervoxel_id)
    # print(root_id, type(root_id))
    # latest_root_id = cggraph.get_latest_roots(root_id)
    # print(latest_root_id, type(latest_root_id))
    
    return root_id, supervoxel_id

In [5]:
import json
from caveclient.base import BaseEncoder, handle_response

def custom_do_merge(supervoxels, coords, resolution, params={
        "priority": False,
        "chebyshev_distance": 10
    }) -> None:
    
    endpoint_mapping = cggraph.default_url_mapping
    url = cggraph._endpoints["do_merge"].format_map(endpoint_mapping)

    data = []
    for svid, coor in zip(supervoxels, coords):
        pos = np.array(coor) * resolution
        row = [svid, pos[0], pos[1], pos[2]]
        data.append(row)
    params = params
    response = cggraph.session.post(
        url,
        data=json.dumps(data, cls=BaseEncoder),
        params=params,
        headers={"Content-Type": "application/json"},
    )
    handle_response(response)

In [5]:
!pwd

/workspaces/localContainer/dist/notebooks


In [12]:
path = "../data/wip/long_edges_fixed"
chunks = pd.read_csv(path)
print(chunks.shape)

(3778, 8)


In [14]:
for index, row in tqdm(chunks.iterrows(), total=chunks.shape[0]):
    if index > -1:
        seg1_id, seg2_id, seg1_x, seg1_y, seg1_z, seg2_x, seg2_y, seg2_z = [*row]
        print(seg1_id, seg2_id, seg1_x, seg1_y, seg1_z, seg2_x, seg2_y, seg2_z)
        seg1_root_id, seg1_supervoxel_id = get_latest_root_id(seg1_x, seg1_y, seg1_z)
        seg2_root_id, seg2_supervoxel_id = get_latest_root_id(seg2_x, seg2_y, seg2_z)
        print(index, seg1_root_id, seg2_root_id)
        if seg1_root_id != seg2_root_id:
            try:
                # cggraph.do_merge(supervoxels=[seg1_supervoxel_id, seg2_supervoxel_id], 
                #                 coords=np.array([[seg1_x, seg1_y, seg1_z], [seg2_x, seg2_y, seg2_z]]),
                #                 resolution=resolution)
                custom_do_merge(supervoxels=[seg1_supervoxel_id, seg2_supervoxel_id], 
                                coords=np.array([[seg1_x, seg1_y, seg1_z], [seg2_x, seg2_y, seg2_z]]),
                                resolution=resolution, 
                                params={
                                    "priority": False,
                                    "chebyshev_distance": 10000
                                })
                print("merge")
                time.sleep(0.5)
            except Exception as e: 
                raise Exception(f"{e}, row: {row}")
                # print("error: ", e)
                # print(index, row)

  0%|          | 0/3778 [00:00<?, ?it/s]

6614076577 8123601364 32409 21780 2152 32359 22020 2394


  0%|          | 1/3778 [00:01<1:09:49,  1.11s/it]

0 864691128623803098 864691128623803098
6675270234 8205199794 32626 20264 2158 32542 20532 2339


Decompressing: 100%|██████████| 1/1 [00:00<00:00, 415.20it/s]


...

100%|█████████▉| 3777/3778 [2:42:58<00:02,  2.76s/it]

30012106575 30012232956 33520 12113 8669 34161 12405 8584


Decompressing: 100%|██████████| 1/1 [00:00<00:00, 1763.05it/s]


3777 864691128630390862 864691128626070444
merge


100%|██████████| 3778/3778 [2:43:02<00:00,  2.59s/it]
